In [1]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix

# LinearRegressionLoss

In [2]:
class LeastSquares:
    def __init__(self, X, y, datasetName=None, scale=1.0):
        """
        docstring
        """
        self.n, self.p = X.shape
        self.X, self.y = X, y
        self.datasetName = datasetName
        self.scale = scale

    def __str__(self):
        info = ""
        if self.datasetName is not None:
            info += "Dataset:{:.>48}\n".format(self.datasetName)
        info += "Data Size:{:.>38}n={}, p={}\nLoss Function:{:.>34}LeastSquares\n".format('', self.n, self.p, '')
        return info

    def evaluate_function_value(self, weight):
        """

        """
        self.matvec = self.X@weight - self.y
        f = 0.5 * np.sum(self.matvec * self.matvec) / self.n
        return f * self.scale

    def gradient(self):
        """
        need to be called after `evaluate_function_value` to get correct `expterm`
        """
        gradient = self.matvec.T @ self.X / self.n
        return gradient.T * self.scale

    def _prepare_hv_data(self, subgroup_index):
        self.X_subset = self.X[:, subgroup_index]

    def hessian_vector_product_fast(self, v):
        temp = (self.X_subset@v)
        hv = (temp.T@self.X_subset).T / self.n
        return (hv + 1e-8 * v) * self.scale

# Group L1

In [3]:
class GL1:
    def __init__(self, Lambda, group, scale=1.0):
        """
        !!Warning: need `group` be ordered in a consecutive manner, i.e., 
        group: array([1., 1., 1., 2., 2., 2., 3., 3., 3., 3.])
        Then:
        unique_groups: array([1., 2., 3.])
        group_frequency: array([3, 3, 4]))
        """
        self.group = group
        self.Lambda = Lambda * scale
        self.unique_groups, self.group_frequency = np.unique(
            self.group, return_counts=True)
        self.Lambda_group = self.Lambda * np.sqrt(self.group_frequency)
        self.K = len(self.unique_groups)
        self.group_size = -1 * np.ones(self.K)
        p = group.shape[0]
        full_index = np.arange(p)
        starts = []
        ends = []
        data = np.zeros(p)
        row_idx = np.zeros(p)
        col_idx = np.zeros(p)
        for i in range(self.K):
            G_i = full_index[np.where(self.group == self.unique_groups[i])]
            # record the `start` and `end` indices of the group G_i to avoid fancy indexing innumpy
            # in the example above, the start index and end index for G_1 is 0 and 2 respectively
            # since python `start:end` will include `start` and exclude `end`, so we will add 1 to the `end`
            # so the G_i-th block of X is indexed by X[start:end]
            start, end = min(G_i), max(G_i) + 1
            starts.append(start)
            ends.append(end)
            self.group_size[i] = end - start
            data[start:end] = self.Lambda_group[i] ** 2
            row_idx[start:end] = i
            col_idx[start:end] = np.arange(start, end)
        # wrap as np.array for jit compile purpose
        self.starts = np.array(starts)
        self.ends = np.array(ends)
        self.mat = csr_matrix((data, (row_idx, col_idx)), shape=(self.K, p))

    def __str__(self):
        return("Group L1")

    def func(self, X):
        return np.sum(np.sqrt(self.mat@(X ** 2)))

    def grad(self, X, sub_grp_idx):
        g = []
        for i in sub_grp_idx:
            start, end = self.starts[i], self.ends[i]
            X_Gi = X[start:end]
            X_Gi_norm = np.sqrt(np.sum(X_Gi*X_Gi))
            g_Gi = self.Lambda_group[i] * X_Gi / X_Gi_norm
            for e in g_Gi:
                g.append(e)
        return np.array(g)

    def proximal(self, x, gradfx, stepsize):
        prox = np.zeros_like(x)
        gradstep = x - stepsize * gradfx
        for i in range(self.K):
            start, end = self.starts[i], self.ends[i]
            gradstep_Gi = gradstep[start:end]
            prox[start:end] = max(0, 1 - (self.Lambda_group[i] * stepsize/np.sqrt(np.sum(gradstep_Gi*gradstep_Gi))
                                          )
                                  ) * gradstep_Gi
        return prox

    def _prepare_hv_data(self, X, subgroup_index):
        self.hv_data = {}
        start = 0
        for i in subgroup_index:
            start_x, end_x = self.starts[i], self.ends[i]
            XG_i = X[start_x:end_x]
            XG_i_norm = np.sqrt(np.dot(XG_i.T, XG_i))[0][0]
            end = start + end_x - start_x
            self.hv_data[i] = {}
            self.hv_data[i]['XG_i'] = XG_i
            self.hv_data[i]['XG_i_norm'] = XG_i_norm
            self.hv_data[i]['start'] = start
            self.hv_data[i]['end'] = end
            self.hv_data[i]['XG_i_norm_cubic'] = XG_i_norm**3
            start = end

    def hessian_vector_product_fast(self, v, subgroup_index):
        hv = np.empty_like(v)
        for i in subgroup_index:
            start = self.hv_data[i]['start']
            end = self.hv_data[i]['end']
            vi = v[start:end]
            temp = np.matmul(self.hv_data[i]['XG_i'].T, vi)
            hv[start:end] = self.Lambda_group[i] * (1 / self.hv_data[i]['XG_i_norm'] * vi -
                                                    (temp / self.hv_data[i]['XG_i_norm_cubic']) *
                                                    self.hv_data[i]['XG_i'])
        return hv

## create linear regression data

In [4]:
np.random.seed(417)
m, n = 10, 6
A = np.random.randn(m, n)
b = np.random.rand(m, 1)
for i in range(m):
    A[i, i % n:i % n+5] = 0
Asparse = coo_matrix(A)
with open("lsMatrix.txt", 'w') as f:
    f.write(f"{m} {n} {Asparse.nnz}\n")
    for idx in range(len(Asparse.row)):
        i = Asparse.row[idx]
        j = Asparse.col[idx]
        f.write(f"{i} {j} {A[i,j]:+3.9e}\n")
with open("lsLabel.txt", 'w') as f:
    f.write(f"{m}\n")
    for idx in range(len(b)):
        f.write(f"{b[idx,0]:+3.9e}\n")

# unscaled version

In [5]:
# unscaled version
f = LeastSquares(A, b, scale=1.0)
x = np.array([0.0, 0.0, 1.1, 0.0, 2.2, 3.3]).reshape(-1, 1)
cols = [2, 3, 4, 5]
v = np.array([i*1.1 for i in range(len(cols))]).reshape(-1,1)

# test for f
ffun = f.evaluate_function_value(x)
fgrad = f.gradient()
f._prepare_hv_data(cols)
fHv = f.hessian_vector_product_fast(v)
print('ffun:', ffun)
print('fgrad:',fgrad.T)
print('fHv:',fHv.T)

with open("ffun.txt", 'w') as file:
    file.write(f"{1}\n")
    file.write(f"{ffun:+3.9e}\n")
with open("fgrad.txt", 'w') as file:
    file.write(f"{fgrad.shape[0]}\n")
    for idx in range(len(fgrad)):
        file.write(f"{fgrad[idx,0]:+3.9e}\n")
with open("fHv.txt", 'w') as file:
    file.write(f"{fHv.shape[0]}\n")
    for idx in range(len(fHv)):
        file.write(f"{fHv[idx,0]:+3.9e}\n")        

ffun: 1.396288349045707
fgrad: [[ 0.4883544  -0.65816869  1.10094573  0.11615631  0.39778551  0.16850084]]
fHv: [[0.47853925 0.05719982 0.24964918 0.13204143]]


In [23]:

r = GL1(1.23, np.array([0, 0, 1, 1, 1, 2]), scale=1.0)
rfun = r.func(x)
rgrad = r.grad(x, [1, 2])
rprox1 = r.proximal(x, x + 0.1, 0.2)
rprox2 = r.proximal(x, x + 2.0, 0.2)
r._prepare_hv_data(x, [1, 2])
rHv = r.hessian_vector_product_fast(v, [1, 2])
trueProx = r.proximal(x, fgrad, 0.2)
print('rfun:', rfun)
print('rgrad:',rgrad.T)
print('rgrad:',rgrad.T)
print('rprox1:',rprox1.T)
print('rprox2:',rprox2.T)
print('rHv:',rHv.T)
print('trueProx:',trueProx.T)


with open("rfun.txt", 'w') as file:
    file.write(f"{1}\n")
    file.write(f"{rfun:+3.9e}\n")
with open("rgrad.txt", 'w') as file:
    file.write(f"{rgrad.shape[0]}\n")
    for idx in range(len(rgrad)):
        file.write(f"{rgrad[idx,0]:+3.9e}\n")
with open("rprox1.txt", 'w') as file:
    file.write(f"{rprox1.shape[0]}\n")
    for idx in range(len(rprox1)):
        file.write(f"{rprox1[idx,0]:+3.9e}\n")
with open("rprox2.txt", 'w') as file:
    file.write(f"{rprox2.shape[0]}\n")
    for idx in range(len(rprox2)):
        file.write(f"{rprox2[idx,0]:+3.9e}\n")
with open("rHv.txt", 'w') as file:
    file.write(f"{rHv.shape[0]}\n")
    for idx in range(len(rHv)):
        file.write(f"{rHv[idx,0]:+3.9e}\n")
with open("trueProx.txt", 'w') as file:
    file.write(f"{trueProx.shape[0]}\n")
    for idx in range(len(trueProx)):
        file.write(f"{trueProx[idx,0]:+3.9e}\n")

rfun: 9.299146467418636
rgrad: [[0.9527539  0.         1.90550781 1.23      ]]
rgrad: [[0.9527539  0.         1.90550781 1.23      ]]
rprox1: [[-0.         -0.          0.67121747 -0.01560971  1.35804466  2.374     ]]
rprox2: [[-0.154      -0.154       0.34334895 -0.28612413  0.97282203  1.994     ]]
rHv: [[-0.76220312  0.9527539   0.38110156  0.        ]]
trueProx: [[-0.          0.          0.71652698 -0.01891978  1.72691045  3.02029983]]


In [25]:
stepsize_ls = 1.0
stepsize_prox = 0.2
bak = 0
d = trueProx - x
directional_derivative = - np.dot(d.T,d) / stepsize_prox
Fx = f.evaluate_function_value(x) + r.func(x)
while True:
    x_trail = x + stepsize_ls * d
    Ftrial = f.evaluate_function_value(x_trail) + r.func(x_trail)
    lhs = Ftrial - Fx
    rhs = 1e-10 * stepsize_ls * directional_derivative
    sufficient_decrease = lhs <= rhs
    if sufficient_decrease:
        print(f"stepsize:{stepsize_ls} | x_trail:{x_trail.T} | bak:{bak} ")
        break
    if bak > 100:
        break
    bak += 1
    stepsize_ls *= 0.5
with open("backtrack_trial_iterates.txt", 'w') as file:
    file.write(f"{x_trail.shape[0]}\n")
    for idx in range(len(x_trail)):
        file.write(f"{x_trail[idx,0]:+3.9e}\n")    

stepsize:1.0 | x_trail:[[ 0.          0.          0.71652698 -0.01891978  1.72691045  3.02029983]] | bak:0 


# scaled version

In [8]:
f = LeastSquares(A, b, scale=0.09083100)
x = np.array([0.0, 0.0, 1.1, 0.0, 2.2, 3.3]).reshape(-1, 1)
cols = [2, 3, 4, 5]
v = np.array([i*1.1 for i in range(len(cols))]).reshape(-1,1)

# test for f
ffun = f.evaluate_function_value(x)
fgrad = f.gradient()
f._prepare_hv_data(cols)
fHv = f.hessian_vector_product_fast(v)
print('ffun:', ffun)
print('fgrad:',fgrad.T)
print('fHv:',fHv.T)

with open("ffun_scaled.txt", 'w') as file:
    file.write(f"{1}\n")
    file.write(f"{ffun:+3.9e}\n")
with open("fgrad_scaled.txt", 'w') as file:
    file.write(f"{fgrad.shape[0]}\n")
    for idx in range(len(fgrad)):
        file.write(f"{fgrad[idx,0]:+3.9e}\n")
with open("fHv_scaled.txt", 'w') as file:
    file.write(f"{fHv.shape[0]}\n")
    for idx in range(len(fHv)):
        file.write(f"{fHv[idx,0]:+3.9e}\n")

ffun: 0.1268262670321706
fgrad: [[ 0.04435772 -0.05978212  0.1         0.01055059  0.03613126  0.0153051 ]]
fHv: [[0.0434662  0.00519552 0.02267588 0.01199346]]


In [9]:
r = GL1(1.23, np.array([0, 0, 1, 1, 1, 2]), scale=0.09083100)
rfun = r.func(x)
rgrad = r.grad(x, [1, 2])
r._prepare_hv_data(x, [1, 2])
rHv = r.hessian_vector_product_fast(v, [1, 2])
trueProx = r.proximal(x, fgrad, 0.2)
print('rfun:', rfun)
print('rgrad:',rgrad.T)
print('rHv:',rHv.T)
print('trueProx:',trueProx.T)


with open("rfun_scaled.txt", 'w') as file:
    file.write(f"{1}\n")
    file.write(f"{rfun:+3.9e}\n")
with open("rgrad_scaled.txt", 'w') as file:
    file.write(f"{rgrad.shape[0]}\n")
    for idx in range(len(rgrad)):
        file.write(f"{rgrad[idx,0]:+3.9e}\n")
with open("rHv_scaled.txt", 'w') as file:
    file.write(f"{rHv.shape[0]}\n")
    for idx in range(len(rHv)):
        file.write(f"{rHv[idx,0]:+3.9e}\n")
with open("trueProx_scaled.txt", 'w') as file:
    file.write(f"{trueProx.shape[0]}\n")
    for idx in range(len(trueProx)):
        file.write(f"{trueProx[idx,0]:+3.9e}\n")                

rfun: 0.844650772782102
rgrad: [[0.08653959 0.         0.17307918 0.11172213]]
rgrad: [[0.08653959 0.         0.17307918 0.11172213]]
rHv: [[-0.06923167  0.08653959  0.03461584  0.        ]]
trueProx: [[-0.00000000e+00  0.00000000e+00  1.06289997e+00 -2.07670848e-03
   2.15805476e+00  3.27459455e+00]]


In [11]:
0.5 ** 100

7.888609052210118e-31